# AnalogSignalArray Tutorial
`AnalogSignalArrayTutorial.ipynb`

## Overview

The `AnalogSignalArray` is used to store fairly-regularly-sampled temporal signals. Ideally the signals should be sampled regularly, but many of the methods handle irregularly sampled data gracefully, and moreover, the `AnalogSignalArray` makes it easy to sanitize an irregularly sampled signal into an easier-to-work-with regularly sampled signal.

Fundamentally, an `AnalogSignalArray` contains a `.time` attribute, and a `.data` attribute, corresponding to the (n,) sample timestamps, in seconds, and the (m, n) signal values (n samples for each of m signals).

```python
# create an AnalogSignalArray with a single signal, with four samples:
asa = nel.AnalogSignalArray(data=[2, 4, 5, 6])

# create an AnalogSignalArray with a single signal, with four samples:
asa = nel.AnalogSignalArray(timestamps=[0, 1, 2, 3],
                            data=[2, 4, 5, 6])

# create an AnalogSignalArray with two signals, each with four samples:
asa = nel.AnalogSignalArray(timestamps=[0, 1, 2, 3],
                            data=[[2, 4, 5, 6], [5, 4, 3, 2]])

# create an AnalogSignalArray with a single signal, with four samples:
asa = nel.AnalogSignalArray(timestamps=[0, 1, 2, 3],
                            data=[2, 4, 5, 6])

# create an AnalogSignalArray with a single signal, with four samples, 
# and an explicit support:
asa = nel.AnalogSignalArray(timestamps=[0, 1, 2, 3],
                            data=[2, 4, 5, 6],
                            support=nel.EpochArray(0,4))

# create an AnalogSignalArray with a single signal, with four samples, 
# and an explicit sampling rate:
asa = nel.AnalogSignalArray(timestamps=[0, 1, 2, 3],
                            data=[2, 4, 5, 6],
                            fs=1)

# create an AnalogSignalArray with a single signal, with ten samples:
asa = nel.AnalogSignalArray(timestamps=[0, 1, 2, 3, 10, 11, 12, 13, 14, 15],
                            data=[1, 1, 1, 1, 2, 2, 2, 2, 2, 2])

# npl.plot(asa, marker='.')
```

`AnalogSignalArray` (m-D)
\begin{equation}
f: \mathbb{R} \to \mathbb{R}^m \quad \bigl(t \mapsto (y_1, y_2, \ldots, y_m)\bigr)
\end{equation}
which has default domain of $\Omega = \mathbb{R}$, and the support can either be specified, or inferred.

If a support is specified (as a collection of intervals, $\{s_1, s_2, \ldots s_k\}$), then the AnalogSignalArray behaves like
\begin{equation}
f: \mathbb{S} \to \mathbb{R}^m \quad \bigl(t \mapsto (y_1, y_2, \ldots, y_m)\bigr)
\end{equation}
where $\mathbb{S} = \bigcup_{i=1}^k s_i$, so that the AnalogSignalArray is **undefined** in $\Omega \backslash \mathbb{S}$.

To be even more explicit about the internal data organization, an `AnalogSignalArray` contains `data` and `time` attributes, where `data` $\in \mathbb{R}^{m \times n}$ where $m$ is the number of signals, and $n$ is the number of samples; `time` is a numpy vector (1-dimensional array) with shape ($n$,), containing the sample times in seconds.

When looking at the `data` matrix itself (as a numpy array, and not a nelpy object), it is convenient to iterate over signals as follows:
```python
for signal in data:
    pass
```
On the other hand, it is inconvenient to plot with matplotlib directly, since matplotlib plots each **column** of a matrix as a single trace. So to plot `data` with matplotlib, we could do
```python
import matplotlib.pyplot as plt
plt.plot(data.T) # plot each signal as a single continuous trace
```
But even this approach ignores the fact that signals are not always continuous, and so nelpy makes our life easier by considering each signal to be composed of different segments, each one of which is assumed to be continuous. In particular, we can use nelpy's plotting as an almost drop-in replacement for matplotlib:
```python
import nelpy.plotting as npl
npl.plot(asa) # plot each signal as a single trace, but respecting the discontinuities in the signal segments
```
As a further demonstration of how we may choose to interact with segments within the signals, consider the following:
```python
import numpy as np
import nelpy as nel

t = np.linspace(0,10,100)

y1 = t**3
y2 = 3*t**2
y3 = 6*t
y4 = 6*np.ones(t.shape)

asa = nel.AnalogSignalArray(np.vstack((y1, y2, y3, y4)), timestamps=t, support=nel.EpochArray([[0,3], [5,10]]))
```
such that `asa` is now an `AnalogSignalArray` with four signals, and two snippets / segments / epochs.

Then, we may get the timestamps and signal data out using the attributes `asa.time` and `asa.data`, but those do not directly tell us which samples are considered contiguous (so plotting with e.g., `plt.plot(asa.time, asa.data.T)` will result in the discontinuities being lost. A slightly better approach might be to use the `_epochtime` and `_epochdata` special objects attached to the `AnalogSignalArray`. Usage is simple enough, e.g.:
```python
import matplotlib.pyplot as plt

# assume we have asa as defined previously
for timestamps, data in zip(asa._epochtime, asa._epochdata):
    plt.plot(timestamps, data.T) # plot each snippet seperately, so that discontinuities are preserved
```
but even though this is better than `plt.plot(timestamps, data.T)`, the signal snippets all have different colors, making it difficult to see which segments correspond to which signals. Again, this is triviel with nelpy, by simple calling `npl.plot(asa)` (try it out!).

Iteration over an `AnalogSignalArray` (and most core temporal nelpy objects, in fact), iterates over the continuous epochs in the object, so that we could also have done
```python
for snippet in asa:
    # snippet is now also an AnalogSignalArray, but with only a single underlying support epoch
    timestamps, data = (snippet.time, snippet.data) # one continuous epoch at a time
```

Finally, we should probably address indexing a little more. However, as we have seen, it is always fairly easy to extract the underlying numpy arrays corresponding to the timestamps and signal data, so that when in doubt, you can always fall back on doing your analyses / manipulation in numpy, and then wrapping that up in a nelpy object again afterwards. But, nelpy does come with many built-in methods to make life easier, including smoothing, filtering, interpolation, differentiation, and so on. So learning about what can be done is certainly worthwhile, especially since many of these seemingly simple operations can quickly become much trickier when the signals have multiple epochs, since scipy and numpy and other common libraries almost always implicitly assume that the arrays that are passed into common methods can be considered contiguous.

At any rate, back to a quick look at indexing. `AnalogSignalArray`s can always be indexed / restricted using EpochArrays, where the resulting `AnalogSignalArray` will be defined on the intersection of its own underlying support (which is also an EpochArray), and the requested EpochArray used for the restriction. For example, still using our `asa` as defined previously, we could do
```python
ep = nel.EpochArray([2,7])
asa_new = asa[ep]
```
but since `asa` had a support of $[0, 3) \bigcup\ [5,10)$, the `new_asa` will be defined on $[2,3)\bigcup\ [5,7)$.

We can also index epochs with integers, like so
```python
asa[0] # asa restricted to first epoch
asa[1] # asa restricted to second epoch
asa[2] # empty AnalogSignalArray, since there wasn't a third epoch
```
and we can extend this syntax to access individual signals:
```python
asa[0,0] # first epoch, first signal
asa[0,1] # first epoch, second signal
asa[:,2:] # all epochs, third and fourth signals
asa[:,[1,3]] # all epochs, second and fourth signals
``` 
and so on. Eventually, we also plan to support samples as the next element in the indexer, such that indexing will be of the form `asa[epoch, signal, sample]`. This form of indexing is consistent across most nelpy object. For example, a `SpikeTrainArray` can be indexed using the same syntax `sta[epoch, unit, ...]`.

Common tricks: resampling, simplify for plots, joining, casting to BinnedSpikeTrainArrays, changing underlying support, get copy-ish object with attached metadata, without actual data. Using `__call__` and `asarray`. Caveats for the advanced: access data with underscore, `__renew__` after modifying object.


`PositionArray` (1D)
\begin{equation}
f: \mathbb{R} \to \mathbb{R} \quad \bigl(t \mapsto x\bigr)
\end{equation}
> special attributes: `x`, `speed`

`PositionArray` (2D)
\begin{equation}
f: \mathbb{R} \to \mathbb{R}^2 \quad \bigl(t \mapsto (x,y)\bigr)
\end{equation}
> special attributes: `x`, `y`, `speed`


`SpikeTrainArray` (N-D)
\begin{equation}
f: \mathbb{Z} \to \mathbb{R}^{n_i} \quad \bigl(u \mapsto (t_1, t_2, \ldots, t_{n_i})\bigr), \text{where $u \in \{1,2,\ldots N\}$ and where $u$ has $n_i$ spikes}
\end{equation}